<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Google Maps - Route optimisation
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Google%20Sheets/Google_Sheets_Get_data.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedbacks</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet #dataframe #google_maps_api #routes_api

**Author:** [Antonio Georgiev](www.linkedin.com/in/antonio-georgiev-b672a325b)

**Description:** This template analyses a given set of waypoints, optimizes the order of visiting them, and outputs a list with the correct order. Thus, making it useful for travelers, who want to visit multiple locations most efficiently.

## Input

### Import library

In [32]:
import naas
import folium
import polyline
from geopy.geocoders import Nominatim
from itertools import permutations
import googlemaps

### Setup API

In [38]:
api_secret_name = "API" # add your API key directly in the API_KEY variable or using naas secret variable
api_key = naas.secret.get(api_secret_name)  # Read API key from the secret variable
gmaps = googlemaps.Client(key=api_key)
origin = "Circonvallazione Salaria, 00199, Rome RM, Italy"
waypoints = [ "Corso Australia, 35136 Padova PD, Italy", "Viale Jacopo dal Verme, 36100 Vicenza, VI, Italy", "Viale Nino Bixio, 1, 31100 Treviso TV, Italy"]
destination = "Milan, Italy"

## Model

### Convert the addresses to coordinates function

In [39]:
def get_coordinates(locations):
    geolocator = Nominatim(user_agent="geoapiExercises")
    coordinates = []
    for location in locations:
        address = geolocator.geocode(location)
        if address:
            coordinates.append((address.latitude, address.longitude))
    return coordinates

### Calculate the total distance function

In [40]:
def calculate_total_distance(gmaps, waypoints):
    total_distance_meters = 0
    for i in range(len(waypoints) - 1):
        from_point = waypoints[i]
        to_point = waypoints[i + 1]
        route = gmaps.directions(from_point, to_point, mode="driving", optimize_waypoints=True)
        if route:
            total_distance_meters += route[0]["legs"][0]["distance"]["value"]
        else:
            raise ValueError(f"Error calculating distance between waypoints {i} and {i+1}")
    return total_distance_meters

### Optimize the waypoints

In [45]:
def optimize_waypoints(gmaps, waypoints):
    min_distance = float("inf")
    optimized_order = None

    for perm in permutations(waypoints):
        waypoints_order = [waypoints[0]] + list(perm) + [waypoints[-1]]
        total_distance = calculate_total_distance(gmaps, waypoints_order)
        if total_distance < min_distance:
            min_distance = total_distance
            optimized_order = waypoints_order

    return optimized_order

In [69]:
optimized_waypoints = optimize_waypoints(gmaps, coordinates)
optimized_coordinates = [(waypoint[0], waypoint[1]) for waypoint in optimized_waypoints]

map_optimized_route = folium.Map(location=optimized_coordinates[0], zoom_start=5)
for i, waypoint in enumerate(optimized_coordinates):
    folium.Marker(waypoint, popup=f"Waypoint {i+1}: {waypoint}").add_to(map_optimized_route)

for i in range(len(optimized_coordinates) - 1):
    from_point = optimized_coordinates[i]
    to_point = optimized_coordinates[i + 1]
    route = gmaps.directions(from_point, to_point, mode="driving", optimize_waypoints=True)
    if route:
        route_polyline = route[0]["overview_polyline"]["points"]
        decoded_points = polyline.decode(route_polyline)
        folium.PolyLine(locations=decoded_points, color='blue').add_to(map_optimized_route)
        for i, waypoint in enumerate(optimized_waypoints):
            location = gmaps.reverse_geocode(waypoint)
            address = location[0]["formatted_address"]
            folium.Marker(waypoint, popup=f"Waypoint {i+1}: {address}").add_to(map_optimized_route)
    else:
        raise ValueError(f"Error calculating route between waypoints {from_point} and {to_point}")

## Output

### Save the updated Google Sheet

In [70]:
print("Optimized Waypoint Order:")
for i, waypoint in enumerate(optimized_waypoints[1:-1]):
    location = gmaps.reverse_geocode(waypoint)
    address = location[0]["formatted_address"]
    print(f"Waypoint {i+1}: {address}")

    # Calculate and display the total distance for the optimized order
total_distance = calculate_total_distance(gmaps, optimized_waypoints)
total_distance_km = total_distance / 1000
print(f"Total Distance: {total_distance_km:.2f} km")
display(map_optimized_route)

Optimized Waypoint Order:
Waypoint 1: Sottopassaggio per Staz. Nomentana, Piazza Addis Abeba, 1, 00199 Roma RM, Italy
Waypoint 2: Str. Pelosa, 4, 35136 Padova PD, Italy
Waypoint 3: Porta S.Tomaso, 31100 Treviso TV, Italy
Waypoint 4: Via Monzambano, 2, 36100 Vicenza VI, Italy
Waypoint 5: P.za del Duomo, 2, 20122 Milano MI, Italy
Total Distance: 845.20 km


In [60]:
map_optimized_route = folium.Map(location=optimized_waypoints[0], zoom_start=5)
for i, waypoint in enumerate(optimized_waypoints):
    location = gmaps.reverse_geocode(waypoint)
    address = location[0]["formatted_address"]
    folium.Marker(waypoint, popup=f"Waypoint {i+1}: {address}").add_to(map_optimized_route)
folium.PolyLine(locations=optimized_waypoints, color='blue').add_to(map_optimized_route)
map_optimized_route